In [ ]:
!nvidia-smi

# Install basic toolkit

In [ ]:
!pip install -qq diffusers==0.14.0 transformers xformers git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip install -qq opencv-contrib-python
!pip install -qq controlnet_aux

In [ ]:
import torch
import os
from huggingface_hub import HfApi
from pathlib import Path
from diffusers.utils import load_image
from PIL import Image, ImageOps
import numpy as np
from transformers import pipeline


from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetPipeline,
    UniPCMultistepScheduler,
)

In [ ]:
image = load_image(
    "https://media.discordapp.net/attachments/1092064974794391644/1194275237538316328/image.png?ex=65afc27d&is=659d4d7d&hm=7bdb56abb8dae3a36e7a7943f3f6b08354f830ee843ef3703aac6e64b06814da&=&format=webp&quality=lossless&width=898&height=874"
)

# Invert the black and white colors of the image, the effect will be better
inverted_image = ImageOps.invert(image)
inverted_image

In [ ]:
image = inverted_image

depth_estimator = pipeline('depth-estimation')
image = depth_estimator(image)['depth']
image = np.array(image)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
control_image = Image.fromarray(image)

control_image.save("control.png")

In [ ]:
# Use ControlNet1.1's depth model
checkpoint = "lllyasviel/control_v11f1p_sd15_depth"
controlnet = ControlNetModel.from_pretrained(checkpoint, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [ ]:
prompt = ["jungle, masterpiece, best quality", "leaves, masterpiece, best quality", "waves, masterpiece, best quality", "cloud, masterpiece, best quality"]
generator = torch.manual_seed(0)
image = pipe(prompt, num_inference_steps=30, generator=generator, image=control_image).images

# image.save('image_out.png')

In [ ]:
# Used to display image
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

image_grid(image, 2, 2)

# Test the effect without color inversion

In [ ]:
image = load_image(
    "https://media.discordapp.net/attachments/1092064974794391644/1194275237538316328/image.png?ex=65afc27d&is=659d4d7d&hm=7bdb56abb8dae3a36e7a7943f3f6b08354f830ee843ef3703aac6e64b06814da&=&format=webp&quality=lossless&width=898&height=874"
)


In [ ]:
prompt = "High detail,(detailed light:0.8),rainwater,Ambient light,3d rendering,realistic,8k,cinema light,(Osmanthus fragrans:1.4),Osmanthus fragrans,Water drop,(rain:1.2),(Fuzzy background:1.2), Above water, Plant in soil, pond"

depth_estimator = pipeline('depth-estimation')
image = depth_estimator(image)['depth']
image = np.array(image)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
control_image = Image.fromarray(image)

control_image.save("control_noinvert.png")

In [ ]:
generator = torch.manual_seed(0)
image = pipe(prompt, num_inference_steps=30, generator=generator, image=control_image).images[0]
image